In [225]:
import warnings
# from sklearn.exceptions import DataConversionWarning
# warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore')

In [227]:
import numpy as np
import pandas as pd
import re

In [229]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet

In [231]:
df = pd.read_csv('news_data.csv')
df.head()

,headline,description,date
0,Gaza’s jour­nal­ists: ‘Tar­gets’ or ‘ca­su­al­...,Gaza’s jour­nal­ists are be­ing killed at an u...,23-12-2023
1,Is­rael or­ders ‘death cor­ri­dor’ evac­u­a­ti...,Hun­dreds of Pales­tini­ans were forced to eva...,23-12-2023
2,"Is­rael’s war on Gaza: List of key events, day 78",UN Se­cu­ri­ty Coun­cil pass­es res­o­lu­tion ...,23-12-2023
3,US ac­cus­es Iran of be­ing ‘deeply in­volved’...,White House says Tehran is pro­vid­ing Yemeni ...,23-12-2023
4,Is­raeli forces con­tin­ue at­tacks amid crit­...,The wa­tered-down res­o­lu­tion boost­ing aid ...,23-12-2023


In [233]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3338 entries, 0 to 3337
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   headline     3338 non-null   object
 1   description  3338 non-null   object
 2   date         3338 non-null   object
dtypes: object(3)
memory usage: 78.4+ KB


In [235]:
df.shape

(3338, 3)

In [237]:
df.describe()

,headline,description,date
count,3338,3338,3338
unique,3331,3337,692
top,Is­raeli forces kill Pales­tin­ian teenag­er i...,As the con­flict be­tween Is­rael and Gaza en­...,17-10-2023
freq,4,2,39


In [239]:
from nltk.stem import WordNetLemmatizer
stopwords=stopwords.words('english')

In [241]:
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [243]:
#decided this after getting the values of avg using tf-idf vectorizer as these words are of not much info
stopwords.extend(['say','ha','wa','amid','abu'])

In [245]:
df.drop(columns=['headline','date'],inplace=True)

In [247]:
df

,description
0,Gaza’s jour­nal­ists are be­ing killed at an u...
1,Hun­dreds of Pales­tini­ans were forced to eva...
2,UN Se­cu­ri­ty Coun­cil pass­es res­o­lu­tion ...
3,White House says Tehran is pro­vid­ing Yemeni ...
4,The wa­tered-down res­o­lu­tion boost­ing aid ...
...,...
3333,Scores of Pales­tini­ans wound­ed af­ter Is­ra...
3334,Amer­i­can po­lit­i­cal lan­guage does not hav...
3335,Pales­tini­ans con­tin­ue sol­i­dar­i­ty prote...
3336,The June 1967 Arab-Is­raeli war last­ed only s...


In [249]:
df.drop_duplicates()

,description
0,Gaza’s jour­nal­ists are be­ing killed at an u...
1,Hun­dreds of Pales­tini­ans were forced to eva...
2,UN Se­cu­ri­ty Coun­cil pass­es res­o­lu­tion ...
3,White House says Tehran is pro­vid­ing Yemeni ...
4,The wa­tered-down res­o­lu­tion boost­ing aid ...
...,...
3333,Scores of Pales­tini­ans wound­ed af­ter Is­ra...
3334,Amer­i­can po­lit­i­cal lan­guage does not hav...
3335,Pales­tini­ans con­tin­ue sol­i­dar­i­ty prote...
3336,The June 1967 Arab-Is­raeli war last­ed only s...


In [251]:
corpus = []

In [253]:
for i in range(0, len(df)):
    df['description'][i]=df['description'][i].replace('\xad', '')

In [255]:
ps = WordNetLemmatizer()
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['description'][i])
    review = review.lower()
    review = review.split()
    review = [ps.lemmatize (word) for word in review]
    review = [word for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    corpus.append(review)

In [256]:
corpus

['gaza journalist killed unprecedented rate plus israel powerful u ally christian zionist',
 'hundred palestinian forced evacuate bureij nuseirat friday rising attack',
 'un security council pass resolution boost aid delivery gaza major update',
 'white house tehran providing yemeni rebel group weapon tactical intelligence',
 'watered resolution boosting aid gaza criticised country nearly meaningless',
 'latest complaint paris based press freedom group asks court hague probe death seven journalist',
 'u abstains resolution lobbied weaken course several day allowing pas',
 'western medium unable ass accurately growing power mid east axis resistance due prejudice',
 'death palestinian child le horrifying death israeli child author tell al jazeera',
 'accountability transferred software system software engineer laura nolan unpacks use ai war',
 'fatima bhutto talk marc lamont hill support gaza muslim world attack palestinian culture',
 'united nation human right chief volker turk basic ri

In [259]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
cv = TfidfVectorizer(ngram_range=(1,1))

In [261]:
corpus2 = cv.fit_transform(corpus)
corpus2

<3338x5748 sparse matrix of type '<class 'numpy.float64'>'
	with 36063 stored elements in Compressed Sparse Row format>

In [263]:
avg=corpus2.mean(axis=0)
avg=pd.DataFrame(avg, columns=cv.get_feature_names_out())

In [265]:
avg=avg.T

In [267]:
avg.head()

,0
abandonbiden,0.000130
abandoned,0.000338
abandonment,0.000105
abbas,0.001295
abd,0.000224


In [269]:
avg=avg.rename(columns={0: 'score'})

In [271]:
avg['word']=avg.index

In [273]:
avg=avg.sort_values('score', ascending=False)

In [275]:
avg[:50]

,score,word
israel,0.037470,israel
israeli,0.036572,israeli
palestinian,0.036111,palestinian
gaza,0.034849,gaza
killed,0.019298,killed
war,0.018581,war
attack,0.017066,attack
hamas,0.016797,hamas
al,0.015907,al
west,0.013536,west


## We've identified key words using unigrams, and by incorporating bigrams, we've gained a deeper understanding of contextual relationships. Now, we can extract relevant parts of sentences from corpus2 based on these important words and use corpus2_bigram to output sentences that are contextually related as related text.

In [287]:
cv_bigram = TfidfVectorizer(ngram_range=(3,3)) 
corpus2_bigram = cv_bigram.fit_transform(corpus)
avg_bigram=corpus2_bigram.mean(axis=0)
avg_bigram=pd.DataFrame(avg_bigram, columns=cv_bigram.get_feature_names_out())
avg_bigram=avg_bigram. T
avg_bigram=avg_bigram. rename (columns={0: 'score'})
avg_bigram['word']=avg_bigram. index
avg_bigram=avg_bigram.sort_values('score', ascending=False)

In [291]:
avg_bigram[:50]

,score,word
occupied west bank,0.007359,occupied west bank
israel hamas war,0.004353,israel hamas war
day main development,0.004160,day main development
israel gaza enters,0.003393,israel gaza enters
conflict israel gaza,0.003393,conflict israel gaza
enters th day,0.003157,enters th day
th day main,0.003091,th day main
gaza enters th,0.002895,gaza enters th
al jazeera journalist,0.002741,al jazeera journalist
update israel hamas,0.002551,update israel hamas


In [293]:
unigrams_list=avg['word'].tolist()
bigrams_list=avg_bigram['word'].tolist()

In [300]:
def convert(lst):
    return ([item.split() for item in lst])

In [315]:
bigrams_split = convert(bigrams_list)

In [316]:
bigrams_split

[['occupied', 'west', 'bank'],
 ['israel', 'hamas', 'war'],
 ['day', 'main', 'development'],
 ['israel', 'gaza', 'enters'],
 ['conflict', 'israel', 'gaza'],
 ['enters', 'th', 'day'],
 ['th', 'day', 'main'],
 ['gaza', 'enters', 'th'],
 ['al', 'jazeera', 'journalist'],
 ['update', 'israel', 'hamas'],
 ['israel', 'war', 'gaza'],
 ['killed', 'israeli', 'force'],
 ['al', 'aqsa', 'mosque'],
 ['jenin', 'refugee', 'camp'],
 ['israeli', 'air', 'raid'],
 ['occupied', 'east', 'jerusalem'],
 ['palestinian', 'killed', 'israeli'],
 ['al', 'shifa', 'hospital'],
 ['raid', 'occupied', 'west'],
 ['un', 'security', 'council'],
 ['israeli', 'air', 'strike'],
 ['journalist', 'shireen', 'akleh'],
 ['occupied', 'palestinian', 'territory'],
 ['israel', 'gaza', 'war'],
 ['killed', 'israeli', 'attack'],
 ['president', 'joe', 'biden'],
 ['killed', 'israeli', 'air'],
 ['palestinian', 'health', 'ministry'],
 ['aqsa', 'mosque', 'compound'],
 ['west', 'bank', 'city'],
 ['israeli', 'air', 'attack'],
 ['palestinian', 

In [317]:
check=pd.DataFrame(columns=['Topic','Sub-Topic'])
check

,Topic,Sub-Topic


In [318]:
for i in unigrams_list:
    count=0
    for j in bigram_split:
        if(count<5) and (i==j[0] or i==j[1] or i==j[2]):
            count+=1
            bigram_words=' '.join(j)
            check=pd.concat([check, pd.concat([pd. Series (i, name='Topic'),pd. Series (bigram_words, name='Sub-Topic')], axis=1)], axis=0)

In [323]:
check

,Topic,Sub-Topic
0,israel,israel hamas war
0,israel,israel gaza enters
0,israel,conflict israel gaza
0,israel,update israel hamas
0,israel,israel war gaza
...,...,...
0,flagged,panama flagged vessel
0,flagged,hurt panama flagged
0,flagged,flagged vessel also
0,swan,swan atlantic unidentified


In [327]:
check_updated=check.groupby(['Topic'], as_index = False, sort=False).agg({'Sub-Topic':', '.join})

In [335]:
# Adjust column width to a larger value
pd.set_option('display.max_colwidth', None)

# Adjust max columns to display (optional)
pd.set_option('display.max_columns', None)

# Adjust display of rows (optional)
pd.set_option('display.max_rows', None)

In [357]:
check_indented = pd.DataFrame(check_updated)
extracted_texts = check_indented.style.set_properties(**{'text-align': 'justify'})
extracted_texts